# AIS Transformer

___
Setup and Configurations

In [13]:
import json
import os
from math import radians, sin, cos, sqrt, atan2

import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, RobustScaler
import joblib

# NOTE: The 'utils.plot_ship_trajectory' function requires an external 'utils.py' file.
# from utils import plot_ship_trajectory 

# Global Constants
WINDOW = 20
HORIZON = 1
STEP = 1
PATIENCE = 5
MIN_DELTA = 1e-4
BATCH_SIZE = 128
EPOCHS = 25
LR = 1e-4
DEVICE = "cpu"  # set to "cuda" if GPU

CSV_PATH = "data/ais_data_5min_clean.csv"

# Physics-aware features:
FEATURES = [
    "Latitude",
    "Longtitude",
    "SOG",
    "COG",
    "vx",
    "vy",
    "dSOG",
    "dCOG"
]

TARGET_DIM = 2  # Δlat, Δlon

# For Visualization
import folium

___
Metrics

In [8]:

# Mean absolute Error for Latitude and Longitude
def compute_mae(pred, true):
    if not isinstance(pred, np.ndarray):
        # Detaches the tensor from the computation graph and stops tracking for gradients
        pred = pred.detach().cpu().numpy().astype(np.float32)
    if not isinstance(true, np.ndarray):
        true = true.detach().cpu().numpy().astype(np.float32)

    # This line performs element-wise subtraction
    err = pred - true

    mae_lat = np.mean(np.abs(err[..., 0]))
    mae_lon = np.mean(np.abs(err[..., 1]))

    return mae_lat, mae_lon

# Root Mean Square Error for Latitude and Longitude
def compute_rmse(pred, true):
    if not isinstance(pred, np.ndarray):
        pred = pred.detach().cpu().numpy().astype(np.float32)
    if not isinstance(true, np.ndarray):
        true = true.detach().cpu().numpy().astype(np.float32)

    err = pred - true

    rmse_lat = np.sqrt(np.mean(err[..., 0] ** 2))
    rmse_lon = np.sqrt(np.mean(err[..., 1] ** 2))

    return rmse_lat, rmse_lon

# Calculates the distance between two geographical points using the Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    '''
    Calculates the great-circle distance between two points (lat1, lon1)
    and (lat2, lon2) using the Haversine formula
    '''
    R = 6371000.0
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2) ** 2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# Apply the haversine distance to every predicted/true coordinate pair
def compute_haversine_metrics(pred_coords, true_coords):
    distances = []
    n = min(len(pred_coords), len(true_coords))

    for i in range(n):
        p = pred_coords[i]
        t = true_coords[i]
        d = haversine(p[0], p[1], t[0], t[1])
        distances.append(d)

    distances = np.array(distances)
    # Calculate Mean Haversine Distance and Haversine RMSE
    mean_haversine = distances.mean()
    rmse_haversine = np.sqrt((distances ** 2).mean())
    return mean_haversine, rmse_haversine

# Mean Absolute Error between the predicted speed and the true speed of a vessel
def compute_speed_error(pred_coords, true_coords, seconds = 60.0):
    speeds_pred = []
    speeds_true = []

    n = min(len(pred_coords) - 1, len(true_coords) - 1)

    for i in range(n):
        dp = haversine(pred_coords[i][0], pred_coords[i][1],
                       pred_coords[i + 1][0], pred_coords[i + 1][1])
        dt = haversine(true_coords[i][0], true_coords[i][1],
                       true_coords[i + 1][0], true_coords[i + 1][1])
        # Speed is calculated as Distance / Time
        speeds_pred.append(dp / seconds)
        speeds_true.append(dt / seconds)
    # Calculates the absolute difference between the predicted and true speeds for all intervals
    return np.mean(np.abs(np.array(speeds_pred) - np.array(speeds_true)))

'''
Quantifies the prediction error in how sharply the vessel turns by comparing the predicted rate
of change of the course (Course Over Ground - COG) against the true rate of change
'''
def compute_turn_rate_error(pred_coords, true_coords):
    # calculates COG between successive points based on the change
    # in latitude (dlat) and longitude (dlon)
    def compute_cog(lat, lon):
        dlat = np.diff(lat)
        dlon = np.diff(lon)
        return np.degrees(np.arctan2(dlon, dlat))

    n = min(len(pred_coords), len(true_coords))
    pred = pred_coords[:n]
    true = true_coords[:n]

    pred_cog = compute_cog(pred[:, 0], pred[:, 1])
    true_cog = compute_cog(true[:, 0], true[:, 1])

    # Ensure cogs have at least two points to compute turn rate
    if len(pred_cog) < 2 or len(true_cog) < 2:
        return 0.0  # Return 0 if not enough data for turn rate

    # The turn rate is the difference between two successive COG values
    # Represents the change in heading
    pred_turn = np.diff(pred_cog)
    true_turn = np.diff(true_cog)

    n2 = min(len(pred_turn), len(true_turn))
    return np.mean(np.abs(pred_turn[:n2] - true_turn[:n2]))

# Presenting all trajectory evaluation metrics
def evaluate_metrics(pred_coords, actual_coords):
    n = min(len(pred_coords), len(actual_coords))
    pred = pred_coords[:n]
    true = actual_coords[:n]
    mae_lat, mae_lon = compute_mae(pred, true)
    mean_hav, rmse_hav = compute_haversine_metrics(pred, true)
    speed_err = compute_speed_error(pred, true, seconds = 300)
    turn_err = compute_turn_rate_error(pred, true)

    print("\n===== METRICS =====")
    print(f"MAE Latitude:     {mae_lat:.6f}")
    print(f"MAE Longitude:    {mae_lon:.6f}")
    print(f"Mean Haversine:   {mean_hav:.3f} m")
    print(f"Haversine RMSE:   {rmse_hav:.3f} m")
    print(f"Speed Error:      {speed_err:.4f} m/s")
    print(f"Turn-Rate Error:  {turn_err:.4f} deg")
    print("===================\n")

    return {
        "mae_lat": mae_lat,
        "mae_lon": mae_lon,
        "mean_haversine": mean_hav,
        "rmse_haversine": rmse_hav,
        "speed_error": speed_err,
        "turn_rate_error": turn_err
    }

# Calculate velocity components and deltas
def compute_physics_features(df):
    """Add vx, vy, dSOG, dCOG features per segment."""
    dfs = []

    for (mmsi, seg), g in df.groupby(["MMSI", "Segment"], observed=True):
        g = g.sort_values("Timestamp").copy()

        # Convert COG to radians
        rad = np.radians(g["COG"].values)

        '''
        Calculates the horizontal (vx) and vertical (vy) velocity components from the ship's
        Speed Over Ground (SOG) and Course Over Ground (COG)
        '''
        g["vx"] = g["SOG"] * np.cos(rad)
        g["vy"] = g["SOG"] * np.sin(rad)

        # Difference (delta) in SOG between the current and previous time step
        g["dSOG"] = g["SOG"].diff().fillna(0)
        # Handle 360-degree wrap-around for COG difference
        dCOG_raw = g["COG"].diff()
        dCOG_wrapped = (dCOG_raw + 540) % 360 - 180
        g["dCOG"] = dCOG_wrapped.fillna(0)

        dfs.append(g)

    return pd.concat(dfs, ignore_index=True)

# PyTorch Haversine (for autoregressive rollout)
def haversine_torch(lat1, lon1, lat2, lon2):
    # Earth radius in meters
    R = 6371000.0

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Convert degrees to radians using torch.deg2rad
    dlat_rad = torch.deg2rad(dlat)
    dlon_rad = torch.deg2rad(dlon)
    lat1_rad = torch.deg2rad(lat1)
    lat2_rad = torch.deg2rad(lat2)

    # Haversine formula using torch functions
    a = torch.sin(dlat_rad / 2) ** 2 + \
        torch.cos(lat1_rad) * torch.cos(lat2_rad) * torch.sin(dlon_rad / 2) ** 2
    c = 2 * torch.atan2(torch.sqrt(a), torch.sqrt(1 - a))

    return R * c

___
Model Definition and Training, Validation and Testing Phases

In [9]:
class TrajectoryDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

'''
Define Transformer Encoder
Attention mechanism allows the model to learn that predicting the next turn might rely heavily on the last few steps of dCOG (change in course) and COG (current course),
while predicting speed might rely more on the current SOG (speed over ground) and the dSOG history
'''
class AISTransformer(nn.Module):
    '''
    Embedding size -> embed_Size
    Attention heads allowing the model to focus on different parts of the input window simultaneously -> num_heads
    Feed-Forward network Dimension (2 layers MLP) -> ff_dim
    '''
    def __init__(self, input_size, embed_size=64, num_heads=2,
                 num_layers=2, ff_dim=128, dropout=0.1):
        super().__init__()

        self.embedding = nn.Linear(input_size, embed_size)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_size,
            nhead=num_heads,
            dim_feedforward=ff_dim,
            dropout=dropout,
            batch_first=True
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.encoder.gradient_checkpointing = True

        # predict 1-step delta at a time (autoregressive decoder)
        self.decoder = nn.Linear(embed_size, TARGET_DIM)

    def forward(self, x):
        """Forward only encodes input window; decoding happens outside."""
        # The input x is (B,W,F) and should be float32, which is ensured by the fix.
        z = self.embedding(x)
        z = self.encoder(z)
        return z[:, -1, :]  # last timestep representation

'''
Implements a ship-based (or group-based) data split, ensuring that all trajectory windows belonging to a specific ship (MMSI)
are placed entirely into one of the three sets: Training, Validation, or Testing
'''
def ship_based_split(meta, train_ratio=0.64, val_ratio=0.16, test_ratio=0.20, seed=42):
    """
    Split ships into:
      - Train: 64%
      - Validation: 16%
      - Test: 20%
    """
    if not np.isclose(train_ratio + val_ratio + test_ratio, 1.0):
        raise ValueError("train_ratio + val_ratio + test_ratio must sum to 1.0")
    np.random.seed(seed)

    ships = np.array(meta["mmsi"].unique())
    np.random.shuffle(ships)

    total = len(ships)

    # Compute boundaries
    cutoff_train = int(total * train_ratio)
    cutoff_val = int(total * (train_ratio + val_ratio))

    # Splits
    train_ships = ships[:cutoff_train]
    val_ships = ships[cutoff_train:cutoff_val]
    test_ships = ships[cutoff_val:]

    split_dict = {
        "train_ships": train_ships.tolist(),
        "val_ships": val_ships.tolist(),
        "test_ships": test_ships.tolist(),
        "ratios": {
            "train_ratio": train_ratio,
            "val_ratio": val_ratio,
            "test_ratio": test_ratio
        },
        "total_unique_ships": total
    }

    with open("ship_splits.json", "w") as f:
        json.dump(split_dict, f, indent=4)

    print("\n============================")
    print(" Ship-based Split Summary ")
    print("============================")
    print(f"Total unique ships: {total}")
    print(f"Train ships ({len(train_ships)}): {train_ships.tolist()}")
    print(f"Val ships   ({len(val_ships)}): {val_ships.tolist()}")
    print(f"Test ships  ({len(test_ships)}): {test_ships.tolist()}")
    print("============================\n")

    train_idx = meta.index[meta["mmsi"].isin(train_ships)].to_numpy()
    val_idx = meta.index[meta["mmsi"].isin(val_ships)].to_numpy()
    test_idx = meta.index[meta["mmsi"].isin(test_ships)].to_numpy()

    return train_idx, val_idx, test_idx, train_ships, val_ships, test_ships

# Generate the dataset
def build_dataset():
    print("Loading CSV...")
    df = pd.read_csv(CSV_PATH)
    df = compute_physics_features(df)

    raw_windows = []
    raw_targets = []
    meta_rows = []

    print("Building raw (unscaled) windows...")
    for (mmsi, seg), g in df.groupby(["MMSI", "Segment"], observed=True):
        g = g.sort_values("Timestamp").reset_index(drop=True)

        if len(g) < WINDOW + HORIZON + 1:
            continue

        lat = g["Latitude"].values
        lon = g["Longtitude"].values
        dlat = lat[1:] - lat[:-1]
        dlon = lon[1:] - lon[:-1]
        deltas = np.stack([dlat, dlon], axis=1)

        feat = g[FEATURES].values

        for i in range(0, len(g) - WINDOW - HORIZON, STEP):
            # For each segment, it slides a window

            X_window_raw = feat[i:i + WINDOW]
            # contains the delta lon and delta lat for the next HORIZON
            future_raw = deltas[i + WINDOW - 1:i + WINDOW - 1 + HORIZON]

            if future_raw.shape[0] != HORIZON:
                continue

            raw_windows.append(X_window_raw)
            raw_targets.append(future_raw)

            meta_rows.append({
                "mmsi": mmsi,
                "segment": seg,
                "last_lat": lat[i + WINDOW - 1],
                "last_lon": lon[i + WINDOW - 1]
            })
    # The collected windows are converted into NumPy arrays
    raw_X = np.array(raw_windows, dtype=np.float32)
    raw_y = np.array(raw_targets, dtype=np.float32)
    meta = pd.DataFrame(meta_rows)

    print(f"Built RAW dataset: X={raw_X.shape}, y={raw_y.shape}, meta={meta.shape}")

    # Ship-based split (done ONCE here)
    train_idx, val_idx, test_idx, train_ships, val_ships, test_ships = ship_based_split(meta)

    # Fit scalers on train
    train_feat = raw_X[train_idx].reshape(-1, raw_X.shape[-1])
    train_deltas = raw_y[train_idx].reshape(-1, 2)

    global_scaler_X = StandardScaler().fit(train_feat)
    global_scaler_y = StandardScaler().fit(train_deltas)

    # Transform entire dataset
    X_scaled = global_scaler_X.transform(raw_X.reshape(-1, raw_X.shape[-1])).reshape(raw_X.shape).astype(np.float32)
    y_scaled = global_scaler_y.transform(raw_y.reshape(-1, 2)).reshape(raw_y.shape).astype(np.float32)

    print("Dataset scaling done. No leakage.")

    segment_scalers = {
        (meta.loc[i, "mmsi"], meta.loc[i, "segment"]): (global_scaler_X, global_scaler_y)
        for i in range(len(meta))
    }

    return (
        X_scaled, y_scaled, meta,
        segment_scalers,
        train_idx, val_idx, test_idx,
        global_scaler_X, global_scaler_y
    )

In [ ]:
# Simulates the future trajectory of a ship over a given horizon by using its own predictions as new inputs for the next time step
def autoregressive_rollout(model, window, scaler_X, scaler_y, horizon=HORIZON, seconds = 60.0):
    device = window.device
    B = window.size(0)

    # scaler stats
    X_mean = torch.tensor(scaler_X.mean_, device=device, dtype=torch.float32)
    X_std  = torch.tensor(scaler_X.scale_, device=device, dtype=torch.float32)
    y_mean = torch.tensor(scaler_y.mean_, device=device, dtype=torch.float32)
    y_std  = torch.tensor(scaler_y.scale_, device=device, dtype=torch.float32)

    pred_list = []

    # Inverse-scale last raw features
    # Get the last known physical state
    last_raw = window[:, -1, :] * X_std + X_mean

    prev_lat = last_raw[:, 0]
    prev_lon = last_raw[:, 1]

    # During training we don't update physics features
    prev_raw = last_raw.clone()

    for _ in range(horizon):
        # Encode window
        z = model(window)
        delta_scaled = model.decoder(z)
        pred_list.append(delta_scaled)

        # Inverse-scale Δlat/Δlon
        delta_real = delta_scaled * y_std + y_mean
        dlat = delta_real[:, 0]
        dlon = delta_real[:, 1]

        # Update pos
        cur_lat = prev_lat + dlat
        cur_lon = prev_lon + dlon

        # Distance & SOG
        # Calculates the physical distance (dist_m) between the previous and current predicted points
        dist_m = haversine_torch(prev_lat, prev_lon, cur_lat, cur_lon)
        SOG = dist_m / seconds # minute AIS interval

        # COG (Course Over Ground)
        dY = cur_lat - prev_lat
        dX = cur_lon - prev_lon
        COG = (torch.rad2deg(torch.atan2(dX, dY)) + 360) % 360

        # Velocity Vectors (vx, vy)
        vx = SOG * torch.cos(torch.deg2rad(COG))
        vy = SOG * torch.sin(torch.deg2rad(COG))

        # Deltas (dSOG, dCOG)
        dSOG = SOG - prev_raw[:, 2]
        dCOG = ((COG - prev_raw[:, 3] + 540) % 360) - 180

        # Create the new feature vector with all calculated values
        new_raw = torch.stack(
            [cur_lat, cur_lon, SOG, COG, vx, vy, dSOG, dCOG],
            dim=1
        )

        # Scale new row
        new_scaled = (new_raw - X_mean) / X_std

        # Slide window
        window = torch.cat([window[:, 1:], new_scaled.unsqueeze(1)], dim=1)

        # Update prev
        prev_lat = cur_lat
        prev_lon = cur_lon
        prev_raw = new_raw

    return torch.stack(pred_list, dim=1), window

def train_model():
    (
        X, y, meta,
        segment_scalers,
        train_idx, val_idx, test_idx,
        global_scaler_X, global_scaler_y
    ) = build_dataset()

    train_ds = TrajectoryDataset(X[train_idx], y[train_idx])
    val_ds = TrajectoryDataset(X[val_idx], y[val_idx])
    test_ds = TrajectoryDataset(X[test_idx], y[test_idx])

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0,
    pin_memory=False)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, num_workers=0,
    pin_memory=False)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, num_workers=0,
    pin_memory=False)

    # Ensure model is on the correct device
    model = AISTransformer(input_size=len(FEATURES)).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    # The Huber Loss is selected. This loss is less sensitive to extreme outliers
    # compared to Mean Squared Error (MSE), making training more robust for noisy data
    criterion = nn.HuberLoss(delta=1.0)

    best_val_loss = float("inf")
    patience_counter = 0

    for epoch in range(EPOCHS):

        model.train()
        total_loss = 0
        total_mae_lat = 0
        total_mae_lon = 0
        total_rmse_lat = 0
        total_rmse_lon = 0

        # The inner loop iterates through all batches (xb, yb) in the training data
        for xb, yb in train_loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            # clears gradients from the previous step
            optimizer.zero_grad()

            # The model generates predictions by running the autoregressive rollout over the input window
            pred_scaled, _ = autoregressive_rollout(
                model, xb,
                global_scaler_X, global_scaler_y,
                horizon=HORIZON,
                seconds=300
            )

            pred_t = pred_scaled.float()  # Ensure float32

            loss = criterion(pred_t, yb)
            mae_lat, mae_lon = compute_mae(pred_t, yb)
            rmse_lat, rmse_lon = compute_rmse(pred_t, yb)

            # calculates the gradients of the loss with respect to all model parameters
            loss.backward()
            # updates the parameters using the calculated gradients
            optimizer.step()

            total_loss += loss.item()
            total_mae_lat += mae_lat
            total_mae_lon += mae_lon
            total_rmse_lat += rmse_lat
            total_rmse_lon += rmse_lon

        n_batches = len(train_loader)
        print(f"\nEpoch {epoch + 1}/{EPOCHS}")
        print(f"Train Loss: {total_loss / n_batches:.6f}")
        print(f"Train MAE(lat): {total_mae_lat / n_batches:.6f}, "
              f"MAE(lon): {total_mae_lon / n_batches:.6f}")

        # disable training features and save memory/time,
        # as gradients are not needed for evaluation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(DEVICE), yb.to(DEVICE)

                pred_scaled, _ = autoregressive_rollout(
                    model, xb,
                    global_scaler_X, global_scaler_y,
                    horizon=HORIZON, 
                    seconds = 300)
                pred_t = pred_scaled.float()

                val_loss += criterion(pred_t, yb).item()

        val_loss /= len(val_loader)
        print(f"Val Loss: {val_loss:.6f}")

        # If the calculated val_loss is significantly better than best_val_loss, the model's state
        # (weights and context) is saved to "ais_model_best.pth", and patience is reset
        if val_loss < best_val_loss - MIN_DELTA:
            best_val_loss = val_loss
            # Save model for best val loss
            torch.save({
                "model_state": model.state_dict(),
                "segment_scalers": segment_scalers,
                "meta": meta,
                "train_idx": train_idx,
                "val_idx": val_idx,
                "test_idx": test_idx,
                "global_scaler_X": global_scaler_X,
                "global_scaler_y": global_scaler_y,
                "X": X, "y": y
            }, "ais_model_best.pth")
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= PATIENCE:
                print("Early stopping.")
                break

    # Load best model for final testing
    if patience_counter >= PATIENCE:
        print("Loading best model for test evaluation...")
        best_ckpt = torch.load("ais_model_best.pth", map_location=DEVICE, weights_only=False)
        model.load_state_dict(best_ckpt["model_state"])

    print("\nEvaluating on test set...")
    '''
    The final model is evaluated once on the completely unseen test set (test_loader),
    providing an unbiased estimate of the model's performance on new ships
    '''
    test_loss = 0
    with torch.no_grad():
        for xb, yb in test_loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            pred_scaled, _ = autoregressive_rollout(
                model, xb,
                global_scaler_X, global_scaler_y,
                horizon=HORIZON,
                seconds = 300
            )
            pred_t = pred_scaled.float()
            test_loss += criterion(pred_t, yb).item()

    test_loss /= len(test_loader)
    print(f"Test Loss: {test_loss:.6f}")

    # Final save of the model (could be the best or the last epoch)
    final_save_path = "ais_model_final.pth"
    torch.save({
        "model_state": model.state_dict(),
        "segment_scalers": segment_scalers,
        "meta": meta,
        "train_idx": train_idx,
        "val_idx": val_idx,
        "test_idx": test_idx,
        "global_scaler_X": global_scaler_X,
        "global_scaler_y": global_scaler_y,
        "X": X, "y": y
    }, final_save_path)

    print(f"Model state saved to {final_save_path}.")

___
Prediction of a ship

In [16]:
# Performs autoregressive prediction for a single starting window (idx), incorporating the model's physics-aware features at every stage
def predict_sample_autoregressive(idx, seconds=60.0):
    """Autoregressive prediction using ALL physics-aware features."""
    try:
        ckpt = torch.load("ais_model_final.pth", map_location=DEVICE, weights_only=False)
    except FileNotFoundError:
        print("Error: 'ais_model_final.pth' not found. Run train_model() first.")
        return np.array([])

    meta = ckpt["meta"]
    X = ckpt["X"]
    
    # Use global scalers from the checkpoint
    scaler_X, scaler_y = ckpt["global_scaler_X"], ckpt["global_scaler_y"]

    # Load the model
    model = AISTransformer(input_size=len(FEATURES)).to(DEVICE)
    model.load_state_dict(ckpt["model_state"])
    model.eval()

    # Initial scaled window (already float32 from build_dataset fix)
    window = X[idx].copy()

    # Last observed actual coordinates
    prev_lat = float(meta.loc[idx, "last_lat"])
    prev_lon = float(meta.loc[idx, "last_lon"])

    # Extract last feature values (scaled → inverse scaled)
    last_feat_scaled = window[-1]
    last_feat = scaler_X.inverse_transform(last_feat_scaled.reshape(1, -1))[0]

    prev_SOG = last_feat[2]
    prev_COG = last_feat[3]

    pred_coords = []

    with torch.no_grad():
        # The function loops HORIZON number of times, generating one future step in each iteration
        for _ in range(HORIZON):
            # Model input: ensure it is float32
            x = torch.tensor(window[np.newaxis, ...], dtype=torch.float32).to(DEVICE)

            # Predict Δlat, Δlon (scaled)
            z = model(x)
            # Take the first (and only) batch item, move to CPU, convert to numpy
            delta_scaled = model.decoder(z)[0].cpu().numpy()

            # Inverse-transform deltas
            delta_real = scaler_y.inverse_transform(delta_scaled.reshape(1, -1))[0]
            dlat, dlon = delta_real

            # Update coordinates (now floats/scalars)
            cur_lat = prev_lat + dlat
            cur_lon = prev_lon + dlon
            pred_coords.append([cur_lat, cur_lon])

            # --- Physics Update for Autoregressive Step (using torch for haversine) ---
            p_lat = torch.tensor(prev_lat, dtype=torch.float32, device=DEVICE)
            p_lon = torch.tensor(prev_lon, dtype=torch.float32, device=DEVICE)
            c_lat = torch.tensor(cur_lat, dtype=torch.float32, device=DEVICE)
            c_lon = torch.tensor(cur_lon, dtype=torch.float32, device=DEVICE)

            # Calculates the physical distance (dist_m) between the previous and current predicted points
            dist_m_tensor = haversine_torch(p_lat, p_lon, c_lat, c_lon)
            dist_m = dist_m_tensor.item()
            # -------------------------------------------------------------------------

            SOG = dist_m /seconds

            dY = cur_lat - prev_lat
            dX = cur_lon - prev_lon
            COG = (np.degrees(np.arctan2(dX, dY)) + 360) % 360

            vx = SOG * np.cos(np.radians(COG))
            vy = SOG * np.sin(np.radians(COG))

            dSOG = SOG - prev_SOG
            dCOG = (COG - prev_COG + 540) % 360 - 180  # normalize turn rate

            # New raw feature row (unscaled)
            new_row_array = np.array([[
                cur_lat, cur_lon, SOG, COG, vx, vy, dSOG, dCOG
            ]], dtype=np.float32)

            # Scale new row for next window
            new_scaled = scaler_X.transform(new_row_array)[0]

            # Slide window
            window = np.vstack([window[1:], new_scaled.astype(np.float32)])

            # Update prev state
            # The predicted position and physics features are updated
            prev_lat, prev_lon = cur_lat, cur_lon
            prev_SOG, prev_COG = SOG, COG

    return np.array(pred_coords)


# creates an interactive HTML map using the folium library to visually compare the entire actual trajectory against the concatenation of all predicted steps
def visualize_full_ship_route(full_actual, pred_future, mmsi, save_path=None):
    if len(full_actual) == 0:
        print("No actual data to plot.")
        return

    m = folium.Map(location=[full_actual[0][0], full_actual[0][1]], zoom_start=9)

    # FULL ACTUAL TRAJECTORY (blue)
    folium.PolyLine(
        locations=[[lat, lon] for lat, lon in full_actual],
        color="blue",
        weight=4,
        opacity=0.9,
        tooltip="Actual Route (Full)"
    ).add_to(m)

    # Start and end markers for actual track
    folium.Marker(
        location=[full_actual[0][0], full_actual[0][1]],
        popup="Start (Actual)",
        icon=folium.Icon(color="green")
    ).add_to(m)

    folium.Marker(
        location=[full_actual[-1][0], full_actual[-1][1]],
        popup="End (Actual)",
        icon=folium.Icon(color="blue")
    ).add_to(m)

    # PREDICTED ROUTE (red)
    if len(pred_future) > 0:
        folium.PolyLine(
            locations=[[lat, lon] for lat, lon in pred_future],
            color="red",
            weight=4,
            opacity=0.9,
            tooltip="Predicted Route"
        ).add_to(m)

    # The map is saved as an interactive HTML file
    if save_path is None:
        save_path = f"ship_{mmsi}_actual_vs_predicted.html"

    m.save(save_path)
    print(f"Map saved to {save_path}")


# Manages the overall evaluation process for all segments belonging to a single ship
def evaluate_full_ship(mmsi):
    try:
        ckpt = torch.load("ais_model_final.pth", map_location=DEVICE, weights_only=False)
    except FileNotFoundError:
        print("Error: 'ais_model_final.pth' not found. Run train_model() first.")
        return np.array([]), np.array([])
    
    # Build model and load weights
    model = AISTransformer(input_size=len(FEATURES)).to(DEVICE)
    model.load_state_dict(ckpt["model_state"])
    model.eval()

    # Load scalers & meta
    meta = ckpt["meta"]
    try:
        df_full = pd.read_csv(CSV_PATH)
    except FileNotFoundError:
        print(f"Error: CSV file not found at {CSV_PATH}. Cannot get actual trajectory.")
        return np.array([]), np.array([])
        
    full_ship_df = df_full[df_full["MMSI"] == mmsi].sort_values("Timestamp")
    # Create the true target trajectory
    full_actual = full_ship_df[["Latitude", "Longtitude"]].to_numpy()

    # Get windows for this ship
    # It filters the dataset metadata (meta) to find all starting input windows that belong to the specified ship
    ship_meta = meta[meta["mmsi"] == mmsi].copy()
    if ship_meta.empty:
        print(f"No windows found for MMSI {mmsi}")
        return full_actual, np.array([])

    # Keep consistent order
    ship_meta["window_index"] = ship_meta.index
    ship_meta_sorted = ship_meta.sort_values(["segment", "window_index"])

    all_pred_coords = []
    for idx in ship_meta_sorted["window_index"]:
        # We exclusively use the correct autoregressive prediction function now
        pred_coords = predict_sample_autoregressive(idx, seconds = 300)
        all_pred_coords.extend(pred_coords.tolist())

    # Returns the complete true trajectory (full_actual) and the complete predicted trajectory (pred_future) for the entire ship's route.
    pred_future = np.array(all_pred_coords)
    return full_actual, pred_future

___
Execution of the model

In [17]:
if __name__ == "__main__":
    try:
        df = pd.read_csv(CSV_PATH)
        print(df.head())
    except FileNotFoundError:
        print(f"Error: CSV file not found at {CSV_PATH}. Cannot proceed with training or evaluation.")
        
    # The MMSI to evaluate. Default is 220464000
    mmsi = 220464000 
    
    # Uncomment the line below to train the corrected model
    train_model()

    # --- Evaluation ---
    print(f"\nEvaluating full trajectory for MMSI: {mmsi}")
    actual, predicted = evaluate_full_ship(mmsi)

    if len(predicted) > 0 and len(actual) > 0:
        metrics = evaluate_metrics(predicted, actual)
        
        # --- Visualization ---
        visualize_full_ship_route(actual, predicted, mmsi)
        print(f"Evaluation of ship with mmsi{mmsi} finished")
    else:
        print("Skipping evaluation and visualization due to missing data or model.")

        MMSI  Segment            Timestamp       SOG   COG  Longtitude  \
0  200000000        1  2025-02-27 00:00:00  6.790661  86.8   11.623667   
1  200000000        1  2025-02-27 00:05:00  6.790661  90.2   11.656255   
2  200000000        1  2025-02-27 00:10:00  6.842105  90.7   11.688977   
3  200000000        1  2025-02-27 00:15:00  6.790661  91.0   11.721940   
4  200000000        1  2025-02-27 00:20:00  6.636328  96.9   11.754877   

    Latitude   distance_m  speed_mps_track  
0  56.123468     0.000000         0.000000  
1  56.122908  2020.798694        33.679978  
2  56.122620  2028.419709        33.806995  
3  56.122575  2043.119472        34.051991  
4  56.122122  2042.136349        34.035606  
Loading CSV...
Building raw (unscaled) windows...
Built RAW dataset: X=(44534, 20, 8), y=(44534, 1, 2), meta=(44534, 4)

 Ship-based Split Summary 
Total unique ships: 279
Train ships (178): [210935000, 231884000, 247386800, 244090800, 257064430, 255915569, 220614000, 248891000, 25720

RuntimeError: DataLoader worker (pid(s) 21760, 6840, 4304, 23700) exited unexpectedly